In [1]:
#importing all lib and dependencies
import numpy as np

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt

import pandas as pd


from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from matplotlib.pyplot import figure
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.cryptocurrencies import CryptoCurrencies
print("everything loaded")


C:\Users\13401\.conda\envs\dl_env2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


everything loaded


In [6]:
#api key: V9Z2AN6WG17PDHZU

config = {
    "alpha_vantage": {
        "key": "V9Z2AN6WG17PDHZU",
        "symbol": "IBM",
        "outputsize": "full",
        "key_adjusted_close": "4. close",
    },
    "alpha_vantage_crypto": {
        "key": "V9Z2AN6WG17PDHZU",
        "symbol": "BTC",
        "market": "USD",  #"CNY"
        "key_close": "4b.close (USD)",
    },
    
    "data": {
        "window_size": 20,
        "tran_split_size": 0.80,
    },
    "model": {
        "input_size": 1, #using only one feature: the close price
        "num_lstm_layers": 2,
        "lstm_size": 32,
        "dropout": 0.2,
    },
    "model_mine":{
        "input_size": 5, #features used: open, close, high, low, volume
        "num_lstm_layers": 3,
        "lstm_size": 128,
        "dropout": 0.05
    },
    "training": {
        "device": "cuda", #cuda or cpu
        "batch_size": 64,
        "num_epoch": 1000,
        "learning_rate": 0.0001,
        "scheduler_step_size": 40,
    }
}

In [8]:
def download_data(config):
    ts = TimeSeries(key=config["alpha_vantage"]["key"], output_format = "pandas")
    data, meta_data = ts.get_daily(config["alpha_vantage"]["symbol"], outputsize = 'full')
    return data

def download_crypto_data(config):
    cc = CryptoCurrencies(key = config["alpha_vantage_crypto"]["key"], output_format = 'pandas')
    data, meta_data = cc.get_digital_currency_daily(symbol = config["alpha_vantage_crypto"]["symbol"], market = config["alpha_vantage_crypto"]["market"])
    return data


In [17]:
def plot_data(pd_data):
    pd_data.plot()
    plt.tight_layout()
    plt.title("daily close value")
    plt.grid()
    plt.show()

stock_data = download_data(config)
crypto_data = download_crypto_data(config)
#stock_data["4. close"].plot()
#crypto_data["4b. close (USD)"].plot()

In [10]:
#normalization:
#mean normalization:

def norm_dataframe_mean(df):
    normalized_df=(df-df.mean())/df.std()
    return normalized_df

#min-max normalization:   
#normalized_df=(df-df.min())/(df.max()-df.min())

def norm_dataframe_minmax(df):
    normalized_df=(df-df.min())/(df.max()-df.min())
    return normalized_df

#norm_dataframe_minmax(stock_data)["4. close"].plot()
#norm_dataframe_mean(stock_data)["4. close"].plot()


stock_data_normed = norm_dataframe_mean(stock_data)
crypto_data_normed = norm_dataframe_mean(crypto_data)

In [69]:
#check this: https://www.analyticsvidhya.com/blog/2021/12/stock-price-prediction-using-lstm/

#data preparation
#    x: the last 20 days price              shape: [length_of_data, 20]
#    Y: the future price (21st day price)   shape: [length_of_data, ]
#    
#    train-test split. done by scipy or pandas. 
#    ratio: train : validation : test  =  7:2:1

stock_data_normed["6. close MA"] = stock_data_normed["4. close"].rolling(window=20).mean() 


In [72]:
stock_data_normed["6. close MA"].shape

(5685,)

In [19]:
from sklearn import preprocessing

In [28]:
scaler = preprocessing.StandardScaler().fit(stock_data)
stock_data_scaled = scaler.transform(stock_data)
print(stock_data_scaled)
#sotck_data_scaled_inversed = scaler.inverse_transform(stock_data_scaled)

[[-0.2124308  -0.22346417 -0.26674658 -0.29282988 -0.20768464]
 [-0.20716793 -0.21820934 -0.21238166 -0.21473764 -0.6678931 ]
 [-0.21270779 -0.20133856 -0.23346194 -0.18344537 -0.31299103]
 ...
 [-0.92679617 -0.95609843 -0.96322774 -0.96962926  1.3047768 ]
 [-0.90242078 -0.93203683 -0.95795767 -0.95744466  1.52648238]
 [-0.85394697 -0.8767228  -0.88362196 -0.90372163  1.05868149]]


In [32]:
#here we define x and y:
# x is the window defined by a hyperparameter window_size,
# y is the moving average in this window.

def prepare_data_x(x, window_size):
    #perform windowing
    n_row = x.shape[0] - window_size + 1
    output = np.lib.stride_tricks.as_strided(x, shape = (n_row, window_size), strides = (x.strides[0], x.strides[0]))
    return output[:-1], output[-1]

def prepare_data_y(x, window_size):
    # # perform simple moving average
    # output = np.convolve(x, np.ones(window_size), 'valid') / window_size

    # use the next day as label
    output = x[window_size:]
    return output



In [33]:
prepare_data_x(stock_data_scaled, 20)

(array([[-0.2124308 , -0.20716793, -0.21270779, ..., -0.07310324,
         -0.042357  ,  0.0268913 ],
        [-0.20716793, -0.21270779, -0.18639344, ..., -0.042357  ,
          0.0268913 , -0.02573741],
        [-0.21270779, -0.18639344, -0.19470324, ...,  0.0268913 ,
         -0.02573741,  0.01636555],
        ...,
        [-0.71711235, -0.74148775, -0.71018752, ..., -1.0755415 ,
         -1.01321804, -0.9664062 ],
        [-0.74148775, -0.71018752, -0.67556337, ..., -1.01321804,
         -0.9664062 , -0.92679617],
        [-0.71018752, -0.67556337, -0.64786406, ..., -0.9664062 ,
         -0.92679617, -0.90242078]]),
 array([-0.67556337, -0.64786406, -0.67390141, -0.56836702, -0.66005176,
        -0.84009731, -0.9785939 , -0.95449549, -0.96142032, -0.92319526,
        -0.92319526, -0.88164629, -0.96142032, -0.94923262, -1.0755415 ,
        -1.01321804, -0.9664062 , -0.92679617, -0.90242078, -0.85394697]))